# **IMPORTACIÓN DE LAS LIBRERÍAS NECESARIAS**

In [ ]:
import collections.abc
import collections
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pylab
import re
import shutil
import tensorflow as tf
import time as time
import wget
import zipfile

from cv2 import dnn_superres
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from time import time


# **FUNCIONES AUXILIARES**

### Función definida para facilitar la creación de directorios de trabajo.

In [ ]:
def create_dir(path):
  try: 
      if not os.path.exists(path):
        os.makedirs(path) 
      else:
        shutil.rmtree(path, ignore_errors=True)
        os.makedirs(path)  
  except OSError: 
      print ('Error creando la carpeta con la ruta ',path,' .')

### Función definida para conocer el tamaño de las imágenes procesadas.

In [ ]:
def know_dimensions(path):
  img = cv2.imread(path)
  height, width, channels = img.shape
  print(height, width, channels)

IMAGE_PATH_VANILLA = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/butterfly1.jpg'
know_dimensions(IMAGE_PATH_VANILLA)

IMAGE_PATH_SR = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/butterfly2.jpg'
know_dimensions(IMAGE_PATH_SR)

### Función definida para cargar una imagen a un numpy array.

In [ ]:
def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

### Se activa la asignación de memoria dinámica de la GPU.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    
import pathlib
import tensorflow as tf
tf.get_logger().setLevel('ERROR')           
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### Funciones definidas para aplicar el modelo pre-entrenado de super-resolución. createsr se ha definido para aplicar sr además de disminución de ruido mientras que createsr1 aplica única y exclusivamente super resolución. en base al estudio realizado, a lo largo del trabajo se utilizará createsr, función la cual aplica disminución de ruido, ya que esta aumenta y mejora el número de detecciones obtenidas por el modelo neuronal convolucional junto con SR.

In [ ]:
def createSR(image_path):
  IMAGE_ORIGINAL_PATH = image_path
  IMAGE_SR_PATH = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0_SR.png'
  PATH_TO_MODEL_DIR = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/MODEL_SR/FSRCNN_x2.pb'
  sr = dnn_superres.DnnSuperResImpl_create()
  image = cv2.imread(IMAGE_ORIGINAL_PATH)
  sr.readModel(PATH_TO_MODEL_DIR)
  sr.setModel("fsrcnn", 2)
  result = sr.upsample(image)
  cv2.imwrite(IMAGE_SR_PATH, result)
  img = cv2.imread('/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0_SR.png')
  dst = cv2.fastNlMeansDenoisingColored(img,None,8,8,7,11)
  cv2.imwrite('/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0D_SR.png', dst) 

def createSR1(image_path):
  IMAGE_ORIGINAL_PATH = image_path
  IMAGE_SR_PATH = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0D_SR.png'
  PATH_TO_MODEL_DIR = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/MODEL_SR/FSRCNN_x2.pb'
  sr = dnn_superres.DnnSuperResImpl_create()
  image = cv2.imread(IMAGE_ORIGINAL_PATH)
  sr.readModel(PATH_TO_MODEL_DIR)
  sr.setModel("fsrcnn", 2)
  result = sr.upsample(image)
  cv2.imwrite(IMAGE_SR_PATH, result)

# **APLICACIÓN DE PROCESOS DE SUPER-RESOLUCIÓN (SR)**

### Ejemplo definido para establecer como hacer uso del modelo de SR a la hora de procesar las imágenes requeridas. En este caso se ha hecho uso del modelo pre-entrenado FSRCNN con un factor de amplicación x2.

In [ ]:
def download_model_SR(model_name):
    data_dir = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/MODEL_SR'
    create_dir(data_dir)
    base_url = 'https://github.com/Saafke/FSRCNN_Tensorflow/blob/master/models/'
    model_file = base_url+ model_name +'.pb?raw=true'
    download_model = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/MODEL_SR/"+model_name+'.pb'
    wget.download(model_file, download_model) 
    return download_model

def download_images(url_path):
    data_dir = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/'
    create_dir(data_dir)
    download_image = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/butterfly1.jpg'
    wget.download(url_path, download_image) 
    return download_image

MODEL_NAME = 'FSRCNN_x2'
PATH_TO_MODEL_DIR = download_model_SR(MODEL_NAME)
IMAGE_PATH = 'https://www.sciencemag.org/sites/default/files/styles/article_main_large/public/butterfly_16x9_0.jpg?itok=jZ3DYvGK'
download_images(IMAGE_PATH)

In [ ]:
MODEL_NAME = 'FSRCNN_x2'
PATH_TO_MODEL_DIR = download_model_SR(MODEL_NAME)

data_dir = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/'
create_dir(data_dir) 
data_dir = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/'
create_dir(data_dir) 

sr = dnn_superres.DnnSuperResImpl_create()
image = cv2.imread('/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/butterfly1.jpg')
sr.readModel(PATH_TO_MODEL_DIR)
sr.setModel("fsrcnn", 2)
result = sr.upsample(image)
cv2.imwrite('/usr/share/Data2/objectos_pequeños/PRINCIPAL/data/butterfly2.jpg', result)

# **APLICACIÓN DEL MODELO DE DETECCIÓN DE OBJETOS**

In [ ]:
path = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/"
path_instalation = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/master.zip"
try:
    shutil.rmtree(path, ignore_errors=True)
    os.mkdir(path)
    path_download = "https://github.com/tensorflow/models/archive/master.zip"
    wget.download(path_download, path_instalation)
    fantasy_zip = zipfile.ZipFile(path_instalation)
    fantasy_zip.extractall(path)
    fantasy_zip.close() 
    os.rename("/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/models-master", "/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/models")
    os.remove(path_instalation)
except OSError:
    print ("El directorio que se pretende borrar no existe" % path)
else:
    print ("Directorio de trabajo generado correctamente %s " % path)
os.chdir(path)

# **GENERACIÓN DEL ESPACIO EN EL CUAL SE ALMACENARÁN LAS IMÁGENES A INFERIR**

In [ ]:
path = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATA"
create_dir(path)
os.chdir(path)

path = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS"
create_dir(path)

# **DESCARGA DEL MODELO**

### Función definida para establecer la descarga del modelo, siendo en este caso Efficientdet D4.

In [ ]:
def download_model(model_name, model_date):
    base_url = 'http://download.tensorflow.org/models/object_detection/tf2/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name,
                                        origin=base_url + model_date + '/' + 
                                        model_file,
                                        untar=True)
    return str(model_dir)

MODEL_DATE = '20200711'
MODEL_NAME = 'efficientdet_d4_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

# **DESCARGA DE LAS ETIQUETAS**

### Función definida para establecer la descarga de las etiquetas, siendo en este caso el labelmap de COCO.

In [ ]:
def download_labels(filename):
    base_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
    label_dir = tf.keras.utils.get_file(fname=filename,
                                        origin=base_url + filename,
                                        untar=False)
    label_dir = pathlib.Path(label_dir)
    return str(label_dir)

LABEL_FILENAME = 'mscoco_complete_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)
print(PATH_TO_LABELS)

/home/ivangarcia/.keras/datasets/mscoco_complete_label_map.pbtxt


# **CARGA DEL MODELO**

In [ ]:
import time
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


# **CARGA DE LAS ETIQUETAS**

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# **EVALUACIÓN DEL MODELO INICIAL**

In [ ]:
matplotlib.use('Agg')

### Array definido para almacenar los tiempos de detección.

In [ ]:
tiempos_procesar_deteccion = []

In [ ]:
def make_RAW_inference(image_path,counter,image_save):
  '''APLICO LA DETECCIÓN DE ELEMENTOS'''
  image_np = load_image_into_numpy_array(image_path)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
  tiempos_procesar_deteccion.append(elapsed_time)
  
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()} 
  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  '''ALMACENO LAS BBOXES,CLASES Y PUNTUACIONES EN SUS RESPECTIVAS LISTAS'''

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
    boxes_filter.append(detections['detection_boxes'][item].tolist())
    scores_filter.append(detections['detection_scores'][item])
    classes_filter.append(detections['detection_classes'][item])

  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

  '''VISUALIZO Y ALMACENO EL RESULTADO OBTENIDO'''

  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        min_score_thresh=.3,
        agnostic_mode=False,
        line_thickness=1)
  plt.axis('off')

  dpi=300
  nombre = counter
  plt.imshow(image_np_with_detections)
  plt.savefig(image_save+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  '''FILTRO LAS DETECCIONES POR UNA PUNTUACIÓN MÍNIMA DADA COMO ENTRADA'''

  min_score_thresh = .3
  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]

  salida = []
  salida.append(true_boxes.tolist())
  salida.append(true_scores.tolist())
  salida.append(true_clases.tolist()) 

  image = Image.open(image_path)
  width, height = image.size
  salida.append(width)
  salida.append(height)

  return salida

### Creo los directorios en los cuales se almacenarán los resultados obtenidos tras aplicar la red neuronal convolucional.

In [ ]:
create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS')
create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR')

### La función make_RAW_inference toma tres parámetros como entrada, la imagen sobre la cual se desea aplicar las detecciones de los elementos provistos por el modelo neuronal convolucional pre-entrenado, el nombre con el cual se almacenará dicho resultado, así como la ruta sobre la cual se desea almacenar los mismos.

In [ ]:
make_RAW_inference('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/1.jpg','1.jpg','/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/')

# **INICIALIZACIÓN DEL DICCIONARIO DE NOMBRES**

### Con el fin de medir posteiormente el mAP (mean Average Precision) obtenido con el evaluador de COCO, es necesario definir previamente un diccionario de nombres con el objetivo de realizar dicha métrica correctamente. En primer lugar se carga el JSON el cual contiene las detecciones con los GT y se realiza una correlación entre el nombre de la imagen y su respectivo id.

In [ ]:
jsonString = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/annotations/instances_default.json'
diccionario_ids = {}

with open(jsonString) as json_file:
    data = json.load(json_file)
    for p3 in data['images']:
        data_s_image = p3['file_name']
        data_s_id = p3['id']
        diccionario_ids[data_s_image] = data_s_id

# **CREACIÓN DE LAS ANOTACIONES CON EL MODELO BASE**

In [ ]:
imagenes_dir = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images'
contenido = os.listdir(imagenes_dir)
counter = 0
result = []
tiempos_procesar = []
out = []
contador_veces = 0

for fichero2 in contenido2:
  '''APLICO UNA EXPRESIÓN REGULAR PARA OBTENER EL NOMBRE DE LA IMAGEN A PROCESAR'''
  x = re.search("[1-9]+[0-9]*", fichero2)
  id = x.group(0)
  image_path_gg = "/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images/{}".format(fichero2)

  '''APLICO LA DETECCIÓN DE ELEMENTOS CON LA FUNCIÓN DEFINIDA ANTERIORMENTE'''
  start_time = time.time()
  salida = make_RAW_inference(image_path_gg,fichero2,'/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/')
  elapsed_time = time.time() - start_time
  tiempos_procesar.append(elapsed_time)

  contador_veces = contador_veces+1
  print("Numero de veces que se ha ejecutado: "+str(contador_veces))
  result = []
  converted_num = int(id)

  '''OBTENGO EL ID CORRESPONDIENTE A DICHA IMAGEN'''
  converted_num = diccionario_ids[fichero2]
  print(fichero2, " ",converted_num)

  width = salida[3]
  height = salida[4]
  boxes2 = []

  for box in salida[0]:
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 

    '''
    TRANSFORMO LAS COORDENADAS INICIALES YA QUE ESTAS DEBEN ESTAR EN EL FORMATO
    DE COCO ADECUADO, SIENDO ESTE (XMIN,YMIN,WIDTH,HEIGHT).
    '''

    box_new = []
    box_new.append(xmin)
    box_new.append(ymin)
    box_new.append(xmax-xmin)
    box_new.append(ymax-ymin)
  
    boxes2.append(box_new)
   

  '''VOY GENERANDO UN JSON CON LAS DETECCIONES OBTENIDAS POR CADA IMAGEN'''
  result.extend(
        [
            {
                "image_id": converted_num,
                "category_id": salida[2][k],
                "bbox": box,
                "score": salida[1][k],
            }
            for k, box in enumerate(boxes2)
        ]
  )
  
  for sample in result:    
        out.append(sample) 

'''ALMACENO EL JSON EN UN ARCHIVO PARA SU POSTERIOR EVALUACIÓN'''

with open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/test_data_normal-video1.json', 'w', encoding='utf-8') as file:
  json.dump(out, file, ensure_ascii=False)

# **EVALUACIÓN DE LAS ANOTACIONES CON EL MODELO RAW**

In [ ]:
pylab.rcParams['figure.figsize'] = (10.0, 8.0)
annType = ['segm','bbox','keypoints']

'''SE ESPECIFICA EL TIPO DE DETECCIÓN SIENDO EN ESTE CASO BBOX'''
annType = annType[1]
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

'''SE LEE EL JSON EL CUAL CONTIENE LAS ANOTACIONES CON LOS GROUND TRUTH (GT)'''
annFile = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video3/annotations/instances_default.json'
cocoGt=COCO(annFile)

'''SE LEE EL JSON DEFINIDO ANTERIORMENTE CON LAS ANOTACIONES OBTENIDAS POR EL MODELO RAW'''
resFile = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/test_data_normal-video1.json'
cocoDt=cocoGt.loadRes(resFile)

dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))

cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds

'''
SE ESTABLECE LA CLASE SOBRE LA CUAL SE DESEA OBTENER EL mAP, SIENDO EN ESTE CASO EL ID 3
CORRESPONDIENTE CON LA CLASE COCHE.
'''

cocoEval.params.catIds = [3] 
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

# **PROPUESTA ESTABLECIDA - APLICACIÓN EXCLUSIVA DE SUPER-RESOLUCIÓN:**

### Variables definidas para almacenar el número de pasadas requeridas por cada imagen, así como el tiempo necesario para su procesamiento.

In [ ]:
array_veces = []
tiempos_procesar2 = []
auxiliar_tiempos = 0
diccionario_tiempos = {}

# **DETECCIÓN CON APLICACIÓN EXCLUSIVA DE SUPER-RESOLUCIÓN (SR):**

### Funciones definidas para la creación del cluster de elementos detectados.

In [ ]:
def rectangle_area(sq):
    '''
    FUNCIÓN DEFINIDA PARA CALCULAR EL ÁREA DE UN RECTÁNGULO.
    '''
    sq_min_h = sq[0]
    sq_max_h = sq[2]
    sq_min_w = sq[1]
    sq_max_w = sq[3]
    
    height = sq_max_h - sq_min_h
    if height < 0:
        return -1
    
    width = sq_max_w - sq_min_w
    if width < 0:
        return -1
    
    return height*width


def bbox_to_square(bbox):
    '''
    FUNCIÓN DEFINIDA PARA ESTABLECER EL FORMATO ADECUADO DE LAS BBOX
    AL SIGUIENTE -> [XMIN,YMIN.XMAX,YMAX]
    '''
    ymin,xmin,ymax,xmax = bbox
    bbox_adecuada = [ymin,xmin,ymax,xmax]
    return bbox_adecuada


def obtener_objeto_mas_confiable_idx(lists):
    '''
    FUNCIÓN DEFINIDA PARA OBTENER EL ELEMENTO CON MAYOR PUNTUACIÓN DE ACUERDO
    A LA LISTA DADA COMO ENTRADA. DEVUELVE EL ÍNDICE DEL OBJETO SELECCIONADO
    Y EL ÍNDICE DE LA LISTA QUE LO CONTENIA.
    '''

    mayor_puntuacion = None
    mayor_puntuacion_objeto_idx = None
    mayor_puntuacion_objeto_lista_idx = None
    for l_idx, l in enumerate(lists):
        _, _, puntuacion = l
        lista_actual_con_mayor_puntuacion = puntuacion.max()
        if not mayor_puntuacion or mayor_puntuacion < lista_actual_con_mayor_puntuacion:
            mayor_puntuacion = lista_actual_con_mayor_puntuacion
            mayor_puntuacion_objeto_idx = np.where(puntuacion==mayor_puntuacion)[0][0]
            mayor_puntuacion_objeto_lista_idx = l_idx

    return mayor_puntuacion_objeto_idx, mayor_puntuacion_objeto_lista_idx


def union_de_dos_rectangulos(sq1, sq2):

    '''
   FUNCIÓN ESTABLECIDA PARA CALCULAR LA UNIÓN DEL ÁREA QUE CONFORMAN DOS RECTÁNGULOS.
   '''

    sq1_area = rectangle_area(sq1)
    sq2_area = rectangle_area(sq2)
    
    return sq1_area + sq2_area - intersecion_entre_dos_rectangulos(sq1, sq2) 


def intersecion_entre_dos_rectangulos(sq1, sq2):

   '''
   FUNCIÓN ESTABLECIDA PARA CALCULAR EL ÍNDICE DE IOU EN BASE AL ÁREA QUE COMPONE
   DOS RECTÁNGULOS.
   '''
   sq1_min_h = sq1[0]
   sq1_max_h = sq1[2]
   sq1_min_w = sq1[1]
   sq1_max_w = sq1[3]
   sq2_min_h = sq2[0]
   sq2_max_h = sq2[2]
   sq2_min_w = sq2[1]
   sq2_max_w = sq2[3]
    
   in_sq_min_h = max(sq1_min_h, sq2_min_h)
   in_sq_max_h = min(sq1_max_h, sq2_max_h)
   in_sq_min_w = max(sq1_min_w, sq2_min_w)
   in_sq_max_w = min(sq1_max_w, sq2_max_w)
    
   area = rectangle_area([in_sq_min_h, in_sq_min_w, in_sq_max_h, in_sq_max_w])
    
   if area != -1:
     return area
   else:
     return 0


def intersection_over_union_usando_cuadrados(sq1, sq2):
    '''
    FUNCIÓN DEFINIDA PARA CALCULAR EL ÍNDICE DE IOU
    '''
    intersecion = intersecion_entre_dos_rectangulos(sq1,sq2)
    union = union_de_dos_rectangulos(sq1,sq2)
    
    return intersecion/union


def intersection_over_union(bbox1, bbox2):
    '''
    FUNCIÓN DEFINIDA PARA ESTABLECER EL ÍNDICE DE IOU EN BASE A DOS BBOXES
    DADOS COMO ENTRADA.

    DEVUELVE EL ÍNDICE OBTENIDO.
    '''

    sq1 = bbox_to_square(bbox1)
    sq2 = bbox_to_square(bbox2)

    return intersection_over_union_usando_cuadrados(sq1, sq2)


def obtener_mismo_objeto(obj, detections_list, threshold, ignore_class = True):

    '''
    FUNCIÓN DEFINIDA PARA OBTENER EL OBJETO MÁS CERCANO DE UNA LISTA DE OBJETOS
    EN BASE A UN ÍNDICE DADO COMO ENTRADA (threshold).

    EL OBJETO ESTÁ FORMADO POR UNA TUPLA (BBOX,CLASE,PUNTUACIÓN)

    IGNORE_CLASS SE DEFINE COMO TRUE PARA IGNORAR CLASES A LA HORA DE ASIGNAR 
    EQUIVALENCIAS.

    DEVUELVE UN ENTERO CON EL ÍNDICE DEL OBJETO CON MAYOR ÍNDICE IOU DE ACUERDO
    CON EL ELEMENTO DADO COMO ENTRADA.
    '''

    lista_bboxes, lista_classes, _ = detections_list
    obj_bbox, obj_class, _ = obj
    mejor_iou = None
    mejor_iou_idx = None

    for idx, (bbox, _class) in enumerate(zip(lista_bboxes, lista_classes)):
        iou = intersection_over_union(obj_bbox, bbox)
        if ignore_class or obj_class == _class:
            if iou > threshold and ((not mejor_iou) or iou > mejor_iou):
                mejor_iou = iou
                mejor_iou_idx = idx

    return mejor_iou_idx


def create_clusters(_lists, threshold):

    '''
    FUNCIÓN DEFINIDA PARA CLEAR EL CLUSTER DE ELEMENTOS DETECTADOS ASIGNADNO EQUIVALENCIAS EN BASE
    A LAS LISTAS DE TUPLAS DADAS COMO ENTRADA DE ACUERDO CON LAS COORDENADAS DE SUS RESPECTIVAS
    BBOXES.

    ARGUMENTOS:
    _lists ESTÁ FORMADO POR UNA LISTA DE TUPLAS. CADA UNA DE ESTAS POSEE LA SIGUIENTE ESTRUCTURA:
    _lists = [(np.array(bbox), np.array(classs), np.array(confidences))]

    DEVUELVE UNA LISTA DE TUPLAS [(np.array(bbox), np.array(classs), np.array(confidences))] 
    EN BASE A LAS COINCIDENCIAS DE CADA OBJETO DETECTADO.
    '''

    lists = _lists.copy()
    lista_vacia_a_eliminar_idx = []
    for idx, _list in enumerate(lists):
        _, classes, _ = _list
        if classes.shape[0] ==0:
            lista_vacia_a_eliminar_idx.append(idx)

    for lista_vacia_idx in reversed(sorted(lista_vacia_a_eliminar_idx)):            
        lists.pop(lista_vacia_idx)

    clusters = []   
    while len(lists) > 0:
        objeto_mas_confiable_idx, lista_objeto_mas_confiable_idx = obtener_objeto_mas_confiable_idx(lists)
        lista_objeto_mas_confiable = lists[lista_objeto_mas_confiable_idx]
        lista_objeto_mas_confiable_bboxes, lista_objeto_mas_confiable_classes, lista_objeto_mas_confiable_confidences = lista_objeto_mas_confiable

        '''CREAMOS UN FILTRO PARA EXCLUIR EL OBJETO DE LAS LISTAS SELECCIONADAS'''
        filtro_idx = lista_objeto_mas_confiable_classes.shape[0]*[True]
        filtro_idx[objeto_mas_confiable_idx] = False

        '''OBTENEMOS EL OBJETO MÁS CONFIABLE Y LO EXCLUIMOS DEL RESTO DE LISTAS'''
        bbox_objeto_mas_confiable = lista_objeto_mas_confiable_bboxes[objeto_mas_confiable_idx,:]
        lista_objeto_mas_confiable_bboxes = lista_objeto_mas_confiable_bboxes[filtro_idx]
        clase_objeto_mas_confiable = lista_objeto_mas_confiable_classes[objeto_mas_confiable_idx]
        lista_objeto_mas_confiable_classes = lista_objeto_mas_confiable_classes[filtro_idx]
        puntuacion_objeto_mas_confiable = lista_objeto_mas_confiable_confidences[objeto_mas_confiable_idx]
        lista_objeto_mas_confiable_confidences = lista_objeto_mas_confiable_confidences[filtro_idx]
        assert lista_objeto_mas_confiable_bboxes.shape[0] == lista_objeto_mas_confiable_classes.shape[0] == lista_objeto_mas_confiable_confidences.shape[0]

        '''ACTUALIZAMOS LAS LISTAS'''
        lists[lista_objeto_mas_confiable_idx] = [lista_objeto_mas_confiable_bboxes, lista_objeto_mas_confiable_classes, lista_objeto_mas_confiable_confidences]

        '''SE GENERA LA TUPLA CON EL OBJETO CON MAYOR PUNTUACIÓN'''
        objeto_mayor_puntuacion = (bbox_objeto_mas_confiable, clase_objeto_mas_confiable, puntuacion_objeto_mas_confiable)

        '''SE INICIALIZA EL CLUSTER DE ELEMENTOS'''
        cluster_bboxes = [bbox_objeto_mas_confiable]
        cluster_classes = [clase_objeto_mas_confiable]
        cluster_confidences = [puntuacion_objeto_mas_confiable]

        lista_vacia_a_eliminar_idx = []
        '''BUSCAMOS AL ELEMENTO SELECCIONADO EN OTRAS LISTAS'''
        for list_idx in range(len(lists)):
            if list_idx != lista_objeto_mas_confiable_idx:
                lista_actual = lists[list_idx]
                
                '''OBTENEMOS EL OBJETO CON LA PUNTUACIÓN MÁS CERCANA DE ACUERDO AL ÍNDICE IOU DEFINIDO'''
                mismo_objeto_lista_actual_idx = obtener_mismo_objeto(objeto_mayor_puntuacion, lista_actual, threshold)
                
                '''
                SI EL ELEMENTO OBTENIDO ES DISTINTO DE NONE, SE INSERTA EN EL CLUSTER Y SE ELIMINA
                DE LA LISTAS.
                '''

                if not mismo_objeto_lista_actual_idx is None:
                    '''OBTENEMOS LA INFORMACIÓN'''
                    lista_actual_bboxes, lista_actual_classes, lista_actual_conficendes = lista_actual

                    '''LA AÑADIMOS AL CLUSTER DE ELEMENTOS'''
                    cluster_bboxes.append(lista_actual_bboxes[mismo_objeto_lista_actual_idx])
                    cluster_classes.append(lista_actual_classes[mismo_objeto_lista_actual_idx])
                    cluster_confidences.append(lista_actual_conficendes[mismo_objeto_lista_actual_idx])

                    '''CREAMOS UN FILTRO PARA ELIMINAR EL OBJETO SELECCIONADO'''
                    lista_actual_filtro_idx = lista_actual_classes.shape[0]*[True]
                    lista_actual_filtro_idx[mismo_objeto_lista_actual_idx] = False

                    '''ACTUALIZAMOS LA INFORMACIÓN DE LA LISTA ACTUAL'''
                    lista_actual_bboxes = lista_actual_bboxes[lista_actual_filtro_idx]
                    lista_actual_classes = lista_actual_classes[lista_actual_filtro_idx]
                    lista_actual_conficendes = lista_actual_conficendes[lista_actual_filtro_idx]
                    
                    '''
                    SI LA LISTA ACTUAL NO ESTÁ VACÍA, LA GUARDAMOS EN CASO CONTRARIO LA AÑADIMOS A 
                    LAS LISTAS PENDIENTES DE ELIMINAR
                    '''
                    if lista_actual_classes.shape[0] != 0:
                        lista_actual = lista_actual_bboxes, lista_actual_classes, lista_actual_conficendes
                        lists[list_idx] = lista_actual
                    else:
                        lista_vacia_a_eliminar_idx.append(list_idx)

        cluster = (np.array(cluster_bboxes), np.array(cluster_classes), np.array(cluster_confidences))
        clusters.append(cluster)

        if lista_objeto_mas_confiable_classes.shape[0] == 0:
            lista_vacia_a_eliminar_idx.append(lista_objeto_mas_confiable_idx)

        for lista_vacia_idx in reversed(sorted(lista_vacia_a_eliminar_idx)):            
            lists.pop(lista_vacia_idx)

    return clusters

# **IMPLEMENTACIÓN DE LA PROPUESTA EXCLUSIVA DE SUPER-RESOLUCIÓN: PRIMERA VERSIÓN (VERSIÓN PRIMITIVA)**

In [ ]:
def make_simple_inference_SR(image,counter,x1,y1,diccionario_final_predicciones,
                                               diccionario_final_scores,
                                               diccionario_final_clases,
                                               width,height,auxiliar_tiempos):
  print('Running super-inference for {}... '.format(image), end='\n')
  
  '''OBTENEMOS LAS DETECCIONES DE LA IMAGEN SR DADA COMO ENTRADA'''
  image_np = load_image_into_numpy_array(image)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
    
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])

  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

  '''FILTRAMOS LAS DETECCIONES POR UN SCORE PREVIAMENTE ESTABLECIDO'''
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  min_score_thresh = .4
  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]

  detections['detection_boxes'] = true_boxes
  detections['detection_classes'] = true_clases
  detections['detection_scores'] = true_scores
  
  '''REALIZAMOS LA TRANSFORMACIÓN DE COORDENADAS'''
  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]

      ymin11 = ((((box_detected[0]*height))+y1)/2)
      xmin11 = ((((box_detected[1]*width))+x1)/2)
      ymax11 = ((((box_detected[2]*height))+y1)/2)
      xmax11 = ((((box_detected[3]*width))+x1)/2)

      coordenadas_good = []
      coordenadas_good.append(ymin11/height)
      coordenadas_good.append(xmin11/width)
      coordenadas_good.append(ymax11/height)
      coordenadas_good.append(xmax11/width)

      '''ALMACENAMOS LA INFORMACIÓN EN EL REPSECTIVO DICCIONARIO PASADO COMO ARGUMENTO'''
      diccionario_final_predicciones.append(coordenadas_good)
      diccionario_final_scores.append(true_scores[i])
      diccionario_final_clases.append(true_clases[i])
    
  
def make_inference_SR(image_path,counter,image_save,contador_veces):
  contador_veces = contador_veces+1
  dpi = 300

  print('Running inference for {}... '.format(image_path), end='\n')

  '''GENERAMOS LA IMAGEN SUPER-RESUELTA'''
  createSR(image_path)

  image_np = load_image_into_numpy_array(image_path)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  '''OBTENEMOS LAS DETECCIONES DE ACUERDO CON LA IMAGEN DADA COMO ENTRADA'''
  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
  
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()} 

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  '''FILTRAMOS LAS DETECCIONES EN BASE A UNA PUNTUACIÓN PREVIAMENTE ESTABLECIDA'''
  min_score_thresh=.2
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]
  
  '''LEEMOS LA IMAGEN SUPER-RESUELTA GENERADA'''
  new_counter = 0
  img = cv2.imread(image_path)
  im = Image.open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0D_SR.png')

  contador_final = 0 
  img__READ = cv2.imread(image_path)
  height, width, channels = img__READ.shape

  diccionario_final_predicciones = []
  diccionario_final_scores = []
  diccionario_final_clases = []
    
  contador_veces = contador_veces+len(true_boxes)
  array_veces.append(contador_veces)
  
  '''
  PARA CADA UNA DE LAS DETECCIONES, CALCULAMOS SU CENTRO Y GENERAMOS UNA 
  NUEVA SUBIMAGEN A PARTIR DE LA SUPER-RESUELTA
  '''
  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
      
      diccionario_final_predicciones.append(coordenadas_good)
      diccionario_final_scores.append(true_scores[new_counter])
      diccionario_final_clases.append(true_clases[new_counter])

      a1 = (xmin+xmax)//2
      a2 = (ymin+ymax)//2

      RECORTEX = int(width/2)
      RECORTEY = int(height/2)
      
      '''
      GENERAMOS LA NUEVA IMAGEN OBTENIENDO PÍXELES A LOS LADOS DEL CENTRO DE LA 
      DETECCIÓN DEL OBJETO.
      '''
      im_crop_outside = im.crop((a1-RECORTEX, a2-RECORTEY, a1+RECORTEX, a2+RECORTEY))

      nombre_sr_2 ="/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/{}.jpg".format(str(i))
      im_crop_outside.save(nombre_sr_2, quality=100)
      
      RECORTEX = int(width/2)
      RECORTEY = int(height/2)  

      'REALIZAMOS UNA NUEVA INFERENCIA POR CADA UNA DE LAS SUBIMAGENES GENERADAS'
      score_class = make_simple_inference_SR(nombre_sr_2,new_counter,a1-RECORTEX,a2-RECORTEY,
                                           diccionario_final_predicciones,
                                           diccionario_final_scores,
                                           diccionario_final_clases,
                                           width,height,counter )
  
      new_counter = new_counter+1

  detections['detection_boxes'] = np.array(diccionario_final_predicciones)
  detections['detection_classes'] = diccionario_final_clases
  detections['detection_scores'] = diccionario_final_scores

  diccionario_final2_predicciones = []
  diccionario_final2_scores = []
  diccionario_final2_clases = []

  puntero = 0
  error = [-1,-1,-1,-1]

  '''
  LLEGADOS A ESTE PUNTO CALCULAMOS EL IOU DE LOS ELEMENTOS CON EL OBJETIVO
  DE ELIMINAR MÚLTIPLES DETECCIONES PARA UN MISMO OBJETO.
  '''
  
  for ia in range(len(diccionario_final_predicciones)):
    coordenadas_A = diccionario_final_predicciones[ia]

    if diccionario_final_predicciones[ia][0]!=-1:
      
      diccionario_final2_predicciones.append(diccionario_final_predicciones[ia])
      diccionario_final2_scores.append(diccionario_final_scores[ia])
      diccionario_final2_clases.append(diccionario_final_clases[ia])

      for ib in range(len(diccionario_final_predicciones)):
        if ia != ib:
          
          coordenadas_B = diccionario_final_predicciones[ib]

          if diccionario_final_predicciones[ib][0]!=-1:
              yminA = max(coordenadas_A[0]*height,coordenadas_B[0]*height)
              xminA = max(coordenadas_A[1]*width,coordenadas_B[1]*width)
              ymaxA = min(coordenadas_A[2]*height,coordenadas_B[2]*height)
              xmaxA = min(coordenadas_A[3]*width,coordenadas_B[3]*width)

              interArea = max(0, ymaxA - yminA + 1) * max(0, xmaxA - xminA + 1)

              boxAArea = (coordenadas_A[2]*height - coordenadas_A[0]*height + 1) * (coordenadas_A[3]*width - coordenadas_A[1]*width + 1)
              boxBArea = (coordenadas_B[2]*height - coordenadas_B[0]*height + 1) * (coordenadas_B[3]*width - coordenadas_B[1]*width + 1)
              
              iou = interArea / float(boxAArea + boxBArea - interArea)

              if iou > 0.1:
                  contador = len(diccionario_final2_predicciones)-1
                  if (diccionario_final2_scores[contador]) < diccionario_final_scores[ib]:
                    
                    contador = len(diccionario_final2_predicciones)-1
                    diccionario_final2_predicciones[contador] = diccionario_final_predicciones[ib]
                    diccionario_final2_scores[contador] = diccionario_final_scores[ib]
                    diccionario_final2_clases[contador] = diccionario_final_clases[ib]

                  diccionario_final_predicciones[ib] = error
            
  detections['detection_boxes'] = np.array(diccionario_final2_predicciones)
  detections['detection_classes'] = diccionario_final2_clases
  detections['detection_scores'] = diccionario_final2_scores
  

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

  '''
  VISUALIZAMOS LA IMAGEN RESULTANTE TRAS APLICAR LA PROPUESTA DE MEJORA 
  EXCLUSIVA CON SUPER-RESOLUCIÓN.
  '''
  
  viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=300,
        min_score_thresh=.3,
        agnostic_mode=False,
        line_thickness=1)
  plt.axis('off')
  plt.imshow(image_np_with_detections)
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  min_score_thresh=.3
  img = cv2.imread(image_path)
  nombre = counter
  plt.savefig('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/'+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()

  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  '''
  DEVOLVEMOS EN LA VARIABLE SALIDA LA LISTA DE BBOXES, CLASES, SCORES, ASÍ COMO
  EL ANCHO Y LARGO DE LA IMAGEN DADA COMO ENTRADA.
  '''

  min_score_thresh = .3
  salida = []
  salida.append(boxes.tolist())
  salida.append(scores[0])
  salida.append(clases_detected) 

  image = Image.open(image_path)
  width, height = image.size

  salida.append(width)
  salida.append(height)
  return salida

In [ ]:
make_inference_SR('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/1.jpg','1.jpg','/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/',0)

# **IMPLEMENTACIÓN DE LA PROPUESTA EXCLUSIVA DE SUPER-RESOLUCIÓN: SEGUNDA VERSIÓN (CLUSTER DE ELEMENTOS)**

In [ ]:
def make_simple_inference_SR(image,counter,x1,y1,width,height,auxiliar_tiempos, LISTA_TRIPLAS):
  print('Running super-inference for {}... '.format(image), end='\n')
  dpi = 300
  image2 = np.array(Image.open(image))
  image_np = load_image_into_numpy_array(image)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  '''OBTENEMOS LAS DETECCIONES PARA LA SUBIMAGEN SUPER-RESUELTA'''
  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
    
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

  
  '''FILTRAMOS POR UNA PUNTUACIÓN MÍNIMA PRE-ESTABLECIDA'''
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  min_score_thresh = .4
  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]


  detections['detection_boxes'] = true_boxes
  detections['detection_classes'] = true_clases
  detections['detection_scores'] = true_scores
    
  '''MOSTRAMOS LAS DETECCIONES OBTENIDAS EN LA SUBIMAGEN Y POSTERIORMENTE LA ALMACENAMOS'''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=1)
  plt.axis('off')
  plt.imshow(image2)

  min_score_thresh=.4
  nombre = counter
  plt.savefig('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/FotoAux/'+str(nombre),  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()
  
  '''CREAMOS LA LISTA QUE CONFORMARÁN LA TUPLA DE DETECCIONES'''
  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]

      ymin11 = ((((box_detected[0]*height))+y1)/2)
      xmin11 = ((((box_detected[1]*width))+x1)/2)
      ymax11 = ((((box_detected[2]*height))+y1)/2)
      xmax11 = ((((box_detected[3]*width))+x1)/2)

      coordenadas_good = []
      coordenadas_good.append(ymin11/height)
      coordenadas_good.append(xmin11/width)
      coordenadas_good.append(ymax11/height)
      coordenadas_good.append(xmax11/width)
        
      LISTA_BOXES_TRIPLAS.append(coordenadas_good)
      LISTA_CLASES_TRIPLAS.append(true_clases[i])
      LISTA_SCORES_TRIPLAS.append(true_scores[i])
      
  
  
  LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS)))
  
  
def make_inference_SR(image_path,counter,image_save,contador_veces):
  print('Running inference for {}... '.format(image_path), end='\n')
  contador_veces = contador_veces+1
  dpi = 300
  image2 = np.array(Image.open(image_path))
  '''CREAMOS LA IMAGEN SUPER-RESUELTA'''
  createSR(image_path)
  image_np = load_image_into_numpy_array(image_path)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]

  '''APLICAMOS LAS DETECCIONES A LA IMAGEN DADA COMO ENTRADA'''

  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
      
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()} 

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  '''FILTRAMOS LAS DETECCIONES EN BASE A UNA PUNTUACIÓN PRE-DEFINIDA'''
  min_score_thresh=.2
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]
  
  new_counter = 0
  img = cv2.imread(image_path)
  im = Image.open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0D_SR.png')

  contador_final = 0 

  img__READ = cv2.imread(image_path)
  height, width, channels = img__READ.shape  
  contador_veces = contador_veces+len(true_boxes)
  array_veces.append(contador_veces)

    
  LISTA_TRIPLAS = []
    

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)

      a1 = (xmin+xmax)//2
      a2 = (ymin+ymax)//2

      RECORTEX = int(width/2)
      RECORTEY = int(height/2)
      
      '''GENERAMOS LA NUEVA SUBIMAGEN A PARTIR DE LA SUPER-RESUELTA'''
      im_crop_outside = im.crop((a1-RECORTEX, a2-RECORTEY, a1+RECORTEX, a2+RECORTEY))

      nombre_sr_2 ="/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/{}.jpg".format(str(i))
      im_crop_outside.save(nombre_sr_2, quality=100)
      
      RECORTEX = int(width/2)
      RECORTEY = int(height/2)  

      'VOLVEMOMS A ESTABLECER LA DETECCIÓN DE OBJETOS EN LA NUEVA SUBIMAGEN GENERADA'
      
      make_simple_inference_SR(nombre_sr_2,new_counter,a1-RECORTEX,a2-RECORTEY,
                                           width,height,counter,LISTA_TRIPLAS )
      
      new_counter = new_counter+1


  puntero = 0
    
  '''GENERAMOS LA LISTA PARA CONFORMAR LA TRIPLA DE LA PASADA INICIAL'''  
  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []
  contador_triplas = 0

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
        
      LISTA_BOXES_TRIPLAS.append(coordenadas_good)
      LISTA_CLASES_TRIPLAS.append(true_clases[contador_triplas])
      LISTA_SCORES_TRIPLAS.append(true_scores[contador_triplas])
    
      
      contador_triplas = contador_triplas + 1
      

  LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS)))

  
  '''OBTENEMOS EL CLUSTER DE ELEMENTOS'''
  cluster_final = create_clusters(LISTA_TRIPLAS, threshold =0.1)

  
  '''
  EN BASE A LA LISTA DE TRUPLAS QUE DEVUELVE LA FUNCIÓN CREATE_CLUSTER,
  SELECCIONAMOS A LOS ELEMENTOS CON MAYOR PUNTUACIÓN.  
  '''

  LISTA_BOXES_OK = []
  LISTA_CLASES_OK = []
  LISTA_SCORES_OK = []
    
  for indexcluster in range(len(cluster_final)):
    tripla = cluster_final[indexcluster]
    
    lista_boxes_lista = tripla[0].tolist()
    lista_clases_lista = tripla[1].tolist()
    lista_scores_lista = tripla[2].tolist()
    
    indice = np.argmax(lista_scores_lista)
    LISTA_BOXES_OK.append(lista_boxes_lista[indice])
    LISTA_CLASES_OK.append(lista_clases_lista[indice])
    LISTA_SCORES_OK.append(lista_scores_lista[indice])
      
            
  detections['detection_boxes'] = np.array(LISTA_BOXES_OK)
  detections['detection_classes'] = np.array(LISTA_CLASES_OK)
  detections['detection_scores'] = np.array(LISTA_SCORES_OK)

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

     
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=300,
      min_score_thresh=.3,
      agnostic_mode=False,
      line_thickness=1)

  display(Image.fromarray(image2))
  plt.axis('off')
  plt.imshow(image2)
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  min_score_thresh=.3
  img = cv2.imread(image_path)
  nombre = counter
  plt.savefig('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/'+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()

  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],

  clases_detected = detections['detection_classes']
  
  '''DEVOLVEMOS EN LA VARIABLE SALIDA LAS BBOXES, CLASES Y PUNTUACIONES FINALES OBTENIDAS'''
  min_score_thresh = .3
  salida = []
  salida.append(boxes.tolist())
  salida.append(scores[0])
  salida.append(clases_detected) 

  image = Image.open(image_path)
  width, height = image.size

  salida.append(width)
  salida.append(height)
  return salida

In [ ]:
make_inference_SR('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/1.jpg','1.jpg','/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/',0)

# **IMPLEMENTACIÓN DE LA PROPUESTA EXCLUSIVA DE SUPER-RESOLUCIÓN: TERCERA VERSIÓN (CLUSTER DE ELEMENTOS+ACELERACIÓN ALGORITMO Bron–Kerbosch)**

In [ ]:
class Node(object):
    def __init__(self, name, coordenadas, puntos):
        self.name = name
        self.neighbors = []
        self.coordenadas = coordenadas
        self.x1x2y1y2 = puntos

    def __repr__(self):
        return self.name
    
def find_cliques(potential_clique=[], remaining_nodes=[], skip_nodes=[], depth=0,lista_cliques=[]):
    if len(remaining_nodes) == 0 and len(skip_nodes) == 0:
        lista_cliques.append(potential_clique)
        return 1
    found_cliques = 0
    for node in remaining_nodes:
        new_potential_clique = potential_clique + [node]
        new_remaining_nodes = [n for n in remaining_nodes if n in node.neighbors]
        new_skip_list = [n for n in skip_nodes if n in node.neighbors]
        found_cliques += find_cliques(new_potential_clique, new_remaining_nodes, new_skip_list, depth + 1,lista_cliques)
        remaining_nodes.remove(node)
        skip_nodes.append(node)
    return found_cliques

'''FUNCIÓN DEFINIDA PARA CALCULAR EL CENTROIDE DE ACUERDO CON UNA SERIE DE PUNTOS'''
def centroid1(points):
    x_coords = [p[0] for p in points]
    y_coords = [p[1] for p in points]
    _len = len(points)
    centroid_x = sum(x_coords)/_len
    centroid_y = sum(y_coords)/_len
    return [centroid_x, centroid_y]

In [ ]:
def make_simple_inference_SR(image,counter,x1,y1,width,height,auxiliar_tiempos, LISTA_TRIPLAS):
  print('Running super-inference for {}... '.format(image), end='\n')
  
  dpi = 300
  image2 = np.array(Image.open(image))
  image_np = load_image_into_numpy_array(image)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]
  
  '''OBTENEMOS LAS DETECCIONES PARA LA SUBIMAGEN SUPER-RESUELTA'''
  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
    
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

  
  '''FILTRAMOS POR UNA PUNTUACIÓN MÍNIMA PRE-ESTABLECIDA'''
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']
  min_score_thresh = .4
  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]


  detections['detection_boxes'] = true_boxes
  detections['detection_classes'] = true_clases
  detections['detection_scores'] = true_scores
    
  '''MOSTRAMOS LAS DETECCIONES OBTENIDAS EN LA SUBIMAGEN Y POSTERIORMENTE LA ALMACENAMOS'''
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      instance_masks=detections.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=1)
  plt.axis('off')
  plt.imshow(image2)

  min_score_thresh=.4
  nombre = counter
  plt.savefig('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/FotoAux/'+str(nombre),  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()
  
  '''CREAMOS LA LISTA QUE CONFORMARÁN LA TUPLA DE DETECCIONES'''
  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]

      ymin11 = ((((box_detected[0]*height))+y1)/2)
      xmin11 = ((((box_detected[1]*width))+x1)/2)
      ymax11 = ((((box_detected[2]*height))+y1)/2)
      xmax11 = ((((box_detected[3]*width))+x1)/2)

      coordenadas_good = []
      coordenadas_good.append(ymin11/height)
      coordenadas_good.append(xmin11/width)
      coordenadas_good.append(ymax11/height)
      coordenadas_good.append(xmax11/width)
        
      LISTA_BOXES_TRIPLAS.append(coordenadas_good)
      LISTA_CLASES_TRIPLAS.append(true_clases[i])
      LISTA_SCORES_TRIPLAS.append(true_scores[i])
      
  
  
  LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS)))
  
  
def make_inference_SR(image_path,counter,image_save,contador_veces):
  print('Running inference for {}... '.format(image_path), end='\n')
  contador_veces = contador_veces+1
  dpi = 300
  image2 = np.array(Image.open(image_path))
  
  '''CREAMOS LA IMAGEN SUPER-RESUELTA'''
  createSR(image_path)
  image_np = load_image_into_numpy_array(image_path)
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]

  '''APLICAMOS LAS DETECCIONES A LA IMAGEN DADA COMO ENTRADA'''

  start_time = time.time()
  detections = detect_fn(input_tensor)
  elapsed_time = time.time() - start_time
      
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()} 

  detections['num_detections'] = num_detections
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  image_np_with_detections = image_np.copy()

  '''FILTRAMOS LAS DETECCIONES EN BASE A UNA PUNTUACIÓN PRE-DEFINIDA'''
  min_score_thresh=.2
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  true_boxes  = boxes[scores[0] > min_score_thresh]
  true_scores = scores[0][scores[0] > min_score_thresh]
  true_clases = clases_detected[scores[0] > min_score_thresh]
  
  new_counter = 0
  img = cv2.imread(image_path)
  im = Image.open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/TensorFlow/DATAPROCESS/0D_SR.png')

  contador_final = 0 

  img__READ = cv2.imread(image_path)
  height, width, channels = img__READ.shape  
  contador_veces = contador_veces+len(true_boxes)
  array_veces.append(contador_veces)

  listado_de_nodos_ok = []    
  LISTA_TRIPLAS = []
    

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
      
      a1 = (xmin+xmax)//2
      a2 = (ymin+ymax)//2

      listado_de_nodos_ok.append(Node(str(i),(a1,a2),(xmin,xmax,ymin,ymax)))

      new_counter = new_counter+1

      '''
      a1 y a2 corresponde con las coordenadas del punto, es por ello por lo que
      debo almacenar estos valores para posteriormente calcular la distancia
      entre los mismos con una formula matematica. Aquella distancia la cual no
      supere un umbral predefinido se establece como vecino del nodo
      '''

  all_nodes = []
  for i in range(len(listado_de_nodos_ok)):
     NODO_SELECCIONADO_1 = listado_de_nodos_ok[i]
     lista_vecinos = []
     all_nodes.append(NODO_SELECCIONADO_1)

     a1 = NODO_SELECCIONADO_1.coordenadas[0]
     a2 = NODO_SELECCIONADO_1.coordenadas[1]

     RECORTEX = int(width/2)
     RECORTEY = int(height/2)

     xmin = a1-RECORTEX
     ymin = a2-RECORTEY
     xmax = a1+RECORTEX
     ymax = a2+RECORTEY
      
     for o in range(len(listado_de_nodos_ok)):
       if i != o:
         NODO_SELECCIONADO_2 = listado_de_nodos_ok[o]
         x1 = NODO_SELECCIONADO_1.coordenadas[0]
         x2 = NODO_SELECCIONADO_2.coordenadas[0]
         y1 = NODO_SELECCIONADO_1.coordenadas[1]
         y2 = NODO_SELECCIONADO_2.coordenadas[1]

         xminb = NODO_SELECCIONADO_2.x1x2y1y2[0]
         xmaxb = NODO_SELECCIONADO_2.x1x2y1y2[1]
         yminb = NODO_SELECCIONADO_2.x1x2y1y2[2]
         ymaxb = NODO_SELECCIONADO_2.x1x2y1y2[3]

        

         if (xminb > xmin and xmaxb < xmax and yminb > ymin and ymaxb < ymax):
           '''AÑADO COMO LISTA DE VECINO'''
           lista_vecinos.append(NODO_SELECCIONADO_2)
     NODO_SELECCIONADO_1.neighbors = lista_vecinos
    

  lista_cliques_ok = []
  total_cliques = find_cliques(remaining_nodes=all_nodes,lista_cliques=lista_cliques_ok)
  
  total_cliques = lista_cliques_ok
    
  contador_veces = contador_veces+len(lista_cliques_ok)
  array_veces.append(contador_veces)


  for ix in range(len(lista_cliques_ok)):
    '''CALCULO EL CENTROIDE DE LOS PUNTOS'''
    cliques_index = total_cliques[ix]
    lista_coordenadas = []
    for ab in range(len(cliques_index)):
     coordenadaxyc = cliques_index[ab].coordenadas
     lista_coordenadas.append(coordenadaxyc)
    centroide = centroid1(lista_coordenadas)

    a1 = centroide[0]
    a2 = centroide[1]

    RECORTEX = int(width/2)
    RECORTEY = int(height/2)
      
    ''''''
    im_crop_outside = im.crop((a1-RECORTEX, a2-RECORTEY, a1+RECORTEX, a2+RECORTEY))
    nombre_sr_332 ="/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/{}.jpg".format(str(ix))
    print(nombre_sr_332)
    im_crop_outside.save(nombre_sr_332, quality=100)

    RECORTEX = int(width/2)
    RECORTEY = int(height/2)

    'VOLVEMOS A ESTABLECER LA DETECCIÓN DE OBJETOS EN LA NUEVA SUBIMAGEN GENERADA'
      
    make_simple_inference_SR(nombre_sr_332,new_counter,a1-RECORTEX,a2-RECORTEY,
                                           width,height,counter,LISTA_TRIPLAS )
      
    new_counter = new_counter+1


  puntero = 0
    
  '''GENERAMOS LA LISTA PARA CONFORMAR LA TRIPLA DE LA PASADA INICIAL'''  
  LISTA_BOXES_TRIPLAS = []
  LISTA_CLASES_TRIPLAS = []
  LISTA_SCORES_TRIPLAS = []
  contador_triplas = 0

  for i in range(len(true_boxes)):
      box_detected = true_boxes[i]
      ymin = int(box_detected[0]*height*2)
      xmin = int(box_detected[1]*width*2)
      ymax = int(box_detected[2]*height*2)
      xmax = int(box_detected[3]*width*2)
    
      ymin11 = (box_detected[0])
      xmin11 = (box_detected[1])
      ymax11 = (box_detected[2])
      xmax11 = (box_detected[3])

      coordenadas_good = []
      coordenadas_good.append(ymin11)
      coordenadas_good.append(xmin11)
      coordenadas_good.append(ymax11)
      coordenadas_good.append(xmax11)
        
      LISTA_BOXES_TRIPLAS.append(coordenadas_good)
      LISTA_CLASES_TRIPLAS.append(true_clases[contador_triplas])
      LISTA_SCORES_TRIPLAS.append(true_scores[contador_triplas])
    
      
      contador_triplas = contador_triplas + 1
      

  LISTA_TRIPLAS.append((np.array(LISTA_BOXES_TRIPLAS),np.array(LISTA_CLASES_TRIPLAS),np.array(LISTA_SCORES_TRIPLAS)))

  
  '''OBTENEMOS EL CLUSTER DE ELEMENTOS'''
  cluster_final = create_clusters(LISTA_TRIPLAS, threshold =0.1)

  
  '''
  EN BASE A LA LISTA DE TRUPLAS QUE DEVUELVE LA FUNCIÓN CREATE_CLUSTER,
  SELECCIONAMOS A LOS ELEMENTOS CON MAYOR PUNTUACIÓN.  
  '''

  LISTA_BOXES_OK = []
  LISTA_CLASES_OK = []
  LISTA_SCORES_OK = []
    
  for indexcluster in range(len(cluster_final)):
    tripla = cluster_final[indexcluster]
    
    lista_boxes_lista = tripla[0].tolist()
    lista_clases_lista = tripla[1].tolist()
    lista_scores_lista = tripla[2].tolist()
    
    indice = np.argmax(lista_scores_lista)
    
    LISTA_BOXES_OK.append(lista_boxes_lista[indice])
    LISTA_CLASES_OK.append(lista_clases_lista[indice])
    LISTA_SCORES_OK.append(lista_scores_lista[indice])
      
            
  detections['detection_boxes'] = np.array(LISTA_BOXES_OK)
  detections['detection_classes'] = np.array(LISTA_CLASES_OK)
  detections['detection_scores'] = np.array(LISTA_SCORES_OK)

  boxes_filter = []
  scores_filter = []
  classes_filter = []

  for item in range(len(detections['detection_boxes'])):
      boxes_filter.append(detections['detection_boxes'][item].tolist())
      scores_filter.append(detections['detection_scores'][item])
      classes_filter.append(detections['detection_classes'][item])


  detections['detection_boxes'] = np.array(boxes_filter)
  detections['detection_classes'] = np.array(classes_filter)
  detections['detection_scores'] = np.array(scores_filter)

     
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image2,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=300,
      min_score_thresh=.3,
      agnostic_mode=False,
      line_thickness=1)

  display(Image.fromarray(image2))
  plt.axis('off')
  plt.imshow(image2)
  
  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],
  clases_detected = detections['detection_classes']

  min_score_thresh=.3
  img = cv2.imread(image_path)
  nombre = counter
  plt.savefig('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/'+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)
  plt.clf()

  boxes = detections['detection_boxes']
  scores = detections['detection_scores'],

  clases_detected = detections['detection_classes']
  
  '''DEVOLVEMOS EN LA VARIABLE SALIDA LAS BBOXES, CLASES Y PUNTUACIONES FINALES OBTENIDAS'''
  min_score_thresh = .3
  salida = []
  salida.append(boxes.tolist())
  salida.append(scores[0])
  salida.append(clases_detected) 

  image = Image.open(image_path)
  width, height = image.size

  salida.append(width)
  salida.append(height)
  return salida


In [ ]:
make_inference_SR('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/1.jpg','1.jpg','/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/',0)

# **CREACIÓN DEL JSON MODIFICADO**

In [ ]:
imagenes_dir2 = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images/'
contenido2 = os.listdir(imagenes_dir2)
imagenes_dir2 = []
counter = 0
result = []
create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/')
create_dir('usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/AUX/')

tiempos_procesamiento2 = []
auxiliar_tiempos
array_veces = []
tiempos_procesar2 = []

out = []
concon = 0
'''RECORRO LAS IMÁGENES ALMACENADAS EN UN DIRECTORIO'''
for fichero2 in contenido2:
  x = re.search("[1-9]+[0-9]*", fichero2)
  id = x.group(0)
  image_path_gg = "/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images/{}".format(fichero2)
  print(image_path_gg)
  contador_veces = 0
  
  start_time = time.time()
  salida = make_inference_SR(image_path_gg,fichero2,'/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/',contador_veces)
  
  end_time = time.time()
  elapsed_time = end_time - start_time
  tiempos_procesamiento2.append(elapsed_time)

  counter = counter+1
  result = []
  '''OBTENGO EL ÍNDICE PERTENECIENTE A DICHA IMAGEN'''
  converted_num = int(id)
  converted_num = diccionario_ids[fichero2]

  width = salida[3]
  height = salida[4]
  boxes2 = []

  '''REALIZO LA TRANSOFORMACIÓN DE COORDENADAS DE ACUERDO CON EL FORMATO DE COCO'''
  for box in salida[0]:
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 

    box_new = []
    box_new.append(xmin)
    box_new.append(ymin)
    box_new.append(xmax-xmin)
    box_new.append(ymax-ymin)
  

    boxes2.append(box_new)
  
  '''LOS ALMACENO EN UN JSON'''
  result.extend(
        [
            {
                "image_id": converted_num,
                "category_id": int(salida[2][k]),
                "bbox": box,
                "score": salida[1][k].astype(float),
            }
            for k, box in enumerate(boxes2)
        ]
  )
  
  for sample in result:    
        out.append(sample)

'''GUARDO EL JSON EN UN ARCHIVO'''
with open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/test_data_modificado-VIDEO2.json', 'w', encoding='utf-8') as file:
  json.dump(out, file, ensure_ascii=False)

# **EVALUACIÓN DE LAS ANOTACIONES MODIFICADAS**

In [ ]:
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

'''CARGO EL JSON CON LOS GROUND TRUTH (GT)'''
annFile = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video3/annotations/instances_default.json'
cocoGt=COCO(annFile)

'''CARGO EL JSON CON LAS DETECCIONES OBTENIDAS CON LA PROPUESTA EXCLUSIVA DE SR'''
resFile = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS_SR/test_data_modificado-VIDEO3.json'
cocoDt=cocoGt.loadRes(resFile)

dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))

cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
'''SELECCIONO LA CLASE 3 CORRESPONDIENTE CON LOS COCHES'''
cocoEval.params.catIds = [3] 
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

# **DIBUJAR LOS GT CON LAS ANOTACIONES**

In [ ]:
def create_image(image_path,counter):
  imga = cv2.imread(image_path)
  nombrenuevo = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/valGT/'+counter
  cv2.imwrite(nombrenuevo, imga)
  return nombrenuevo

def draw_rectangle(image_path,x1,y1,x2,y2):
  x1 = round(x1)
  y1 = round(y1)
  x2 = round(x2)
  y2 = round(y2)

  start_point = (x1, y1) 
  end_point = (x1+x2, y1+y2) 
  color = (255, 0, 0) 
  thickness = 2   

  imga = cv2.imread(image_path)
  image = cv2.rectangle(imga, start_point, end_point, color, thickness) 
  cv2.imwrite(image_path, imga)

In [ ]:
jsonString = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/annotations/instances_default.json'
images_dir_dataprocess = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/valGT/'
create_dir(images_dir_dataprocess)

contenido2 = os.listdir('/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images')
contador = 0
for fichero in contenido2:
  nombre_good = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images/'+fichero 
  id_frame = diccionario_ids[fichero]
  salida = create_image(nombre_good,fichero)
  contador=contador+1
  with open(jsonString) as json_file:
      data = json.load(json_file)
      for p3 in data['annotations']:
          nombre_identificador = p3['image_id']
          if nombre_identificador == id_frame:
            lista = p3['bbox']
            draw_rectangle(salida,lista[0],lista[1],lista[2],lista[3])

# **TRANSFER-LEARNING -> (Fine-Tuning)**

### FUNCIONES AUXILIARES DEFINIDAS PARA LA CREACIÓN DE LOS XML DE FORMA AUTOMÁTICA

In [ ]:
start_ranges = "|".join(
    "[{0}]".format(r)
    for r in [
        "\xC0-\xD6",
        "\xD8-\xF6",
        "\xF8-\u02FF",
        "\u0370-\u037D",
        "\u037F-\u1FFF",
        "\u200C-\u200D",
        "\u2070-\u218F",
        "\u2C00-\u2FEF",
        "\u3001-\uD7FF",
        "\uF900-\uFDCF",
        "\uFDF0-\uFFFD",
    ]
)

NameStartChar = re.compile(r"(:|[A-Z]|_|[a-z]|{0})".format(start_ranges))
NameChar = re.compile(r"(\-|\.|[0-9]|\xB7|[\u0300-\u036F]|[\u203F-\u2040])")

class Node(object):
    entities = [("&", "&amp;"), ("<", "&lt;"), (">", "&gt;")]

    def __init__(self, wrap="", tag="", data=None, iterables_repeat_wrap=True):
        self.tag = self.sanitize_element(tag)
        self.wrap = self.sanitize_element(wrap)
        self.data = data
        self.type = self.determine_type()
        self.iterables_repeat_wrap = iterables_repeat_wrap

        if self.type == "flat" and isinstance(self.data, str):
            for entity, replacement in self.entities:
                self.data = self.data.replace(entity, replacement)

    def serialize(self, indenter):
        wrap = self.wrap
        end, start = "", ""
        if wrap:
            end = "</{0}>".format(wrap)
            start = "<{0}>".format(wrap)

        value, children = self.convert()

        content = ""
        if children:
            if self.type != "iterable":
                content = indenter((c.serialize(indenter) for c in children), wrap)
            else:
                if self.iterables_repeat_wrap:
                    result = []
                    for c in children:
                        content = c.serialize(indenter)
                        if c.type == "flat":
                            result.append(content)
                        else:
                            content = indenter([content], True)
                            result.append("".join((start, content, end)))

                    return indenter(result, False)
                else:
                    result = []
                    for c in children:
                        result.append(c.serialize(indenter))
                    return "".join([start, indenter(result, True), end])

        
        return "".join((start, value, content, end))

    def determine_type(self):
        
        data = self.data
        if isinstance(data, str):
            return "flat"
        elif isinstance(data, collections.abc.Mapping):
            return "mapping"
        elif isinstance(data, collections.abc.Iterable):
            return "iterable"
        else:
            return "flat"

    def convert(self):
        
        val = ""
        typ = self.type
        data = self.data
        children = []

        if typ == "mapping":
            sorted_data = data
            if not isinstance(data, collections.OrderedDict):
                sorted_data = (data)

            for key in sorted_data:
                item = data[key]
                children.append(
                    Node(key, "", item, iterables_repeat_wrap=self.iterables_repeat_wrap)
                )

        elif typ == "iterable":
            for item in data:
                children.append(
                    Node("", self.wrap, item, iterables_repeat_wrap=self.iterables_repeat_wrap,)
                )

        else:
            val = str(data)
            if self.tag:
                val = "<{0}>{1}</{2}>".format(self.tag, val, self.tag)

        return val, children

    @staticmethod
    def sanitize_element(wrap):
        
        if wrap and isinstance(wrap, str):
            if wrap.lower().startswith("xml"):
                wrap = "_" + wrap
            return "".join(
                ["_" if not NameStartChar.match(wrap) else ""]
                + ["_" if not (NameStartChar.match(c) or NameChar.match(c)) else c for c in wrap]
            )
        else:
            return wrap

class Converter(object):
    

    def __init__(self, wrap=None, indent="  ", newlines=True):
        
        self.wrap = wrap
        self.indent = indent
        self.newlines = newlines

    def _make_indenter(self):
        indent = self.indent
        newlines = self.newlines

        if not newlines:
            ret = lambda nodes, wrapped: "".join(nodes)
        else:
            if not indent:
                indent = ""

            def eachline(nodes):
                for node in nodes:
                    for line in node.split("\n"):
                        yield line

            def ret(nodes, wrapped):
                if wrapped:
                    seperator = "\n{0}".format(indent)
                    surrounding = "\n{0}{{0}}\n".format(indent)
                else:
                    seperator = "\n"
                    surrounding = "{0}"
                return surrounding.format(seperator.join(eachline(nodes)))

        return ret

    def build(self, data, iterables_repeat_wrap=True):
        indenter = self._make_indenter()
        return Node(
            wrap=self.wrap, data=data, iterables_repeat_wrap=iterables_repeat_wrap
        ).serialize(indenter)



### FUNCIÓN DEFINIDA PARA TRANSFORMAR UN DICCIONARIO EN UN ARCHIVO XML

In [ ]:
def dict2xml(data, *args, **kwargs):
    return Converter(*args, **kwargs).build(data)

### EJEMPLO DE PRUEBA

In [ ]:

json = {
  "annotation": {
    "folder": "images",
    "filename": "1.jpg",
    "path": "/Users/ivan/Desktop/images/1.jpg",
    "source": {
      "database": "Unknown"
    },
    "size": {
      "width": "640",
      "height": "480",
      "depth": "3"
    },
    "segmented": "0",
    "object": [
      {
        "name": "car",
        "pose": "Unspecified",
        "truncated": "0",
        "difficult": "0",
        "bndbox": {
          "xmin": "302",
          "ymin": "215",
          "xmax": "338",
          "ymax": "237"
        }
      },
      {
        "name": "car",
        "pose": "Unspecified",
        "truncated": "0",
        "difficult": "0",
        "bndbox": {
          "xmin": "417",
          "ymin": "199",
          "xmax": "453",
          "ymax": "222"
        }
      }
    ]
  }
}
print(dict2xml(json))

# **CREACIÓN DE LOS RESPECTIVOS XML**

In [ ]:

imagenes_dir2 = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/IMAGENES'
contenido2 = os.listdir(imagenes_dir2)
imagenes_dir2 = []
counter = 0
result = []
create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/RESULTADOS_SR')

tiempos_procesamiento2 = []
out = []
concon = 0

'''RECORRO LAS IMÁGENES ALMACENADAS EN EL DIRECTORIO DADO COMO ENTRADA'''

for fichero2 in contenido2:
  create_dir('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/AUX')
  concon = concon +1
  x = re.search("[1-9]+[0-9]*", fichero2)
  id = x.group(0)
  image_path_gg = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/IMAGENES/{}".format(fichero2)
  image_path_gg2 = "/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/ARCHIVOS/{}".format(fichero2)
  print(image_path_gg)
  start_time = time()
  
  elapsed_time = time() - start_time
  tiempos_procesamiento2.append(elapsed_time)
  salida = make_inference_SR(image_path_gg,fichero2,'/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/RESULTADOS_SR/')
  print("\n")
  counter = counter+1
  result = []
  
  width = salida[3]
  height = salida[4]

  x = '{"annotation":{"folder": "images","filename": "1.jpg","path": "/Users/ivan/Desktop/images/1.jpg","source": {"database": "Unknown"},"size": {"width": "640","height": "480","depth": "3"},"segmented": "0","object": []}}'
  y = json.loads(x)
  y["annotation"]["folder"] = 'IMAGENES'
  y["annotation"]["filename"] = fichero2
  y["annotation"]["path"] = image_path_gg
  y["annotation"]["size"]["width"] = width
  y["annotation"]["size"]["height"] = height

  shutil.copyfile(image_path_gg, image_path_gg2)
  boxes2 = []

  for box in salida[0]:
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 

    objeto1 = '{"name": "car","pose": "Unspecified","truncated": "0","difficult": "0","bndbox": {"xmin": "302","ymin": "215","xmax": "338","ymax": "237"}}'
    z = json.loads(objeto1)
    z["name"] = "car"
    z["bndbox"]["xmin"] = xmin 
    z["bndbox"]["xmax"] = xmax
    z["bndbox"]["ymin"] = ymin
    z["bndbox"]["ymax"] = ymax

    y['annotation']['object'].append(z)

  nombre_bueno = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/IMAGENES_JSON/'+id+'.json'
  nombre_bueno2 = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/ARCHIVOS/'+id+'.xml'
  print(y)
  with open(nombre_bueno, 'w', encoding='utf-8') as f:
    json.dump(y, f)

  '''hacerlo en xml'''
  with open(nombre_bueno) as json_file:
    data = json.load(json_file)
    xml = dict2xml(data)
    with open(nombre_bueno2, "w") as f:
      f.write(xml)

# **SEPARACIÓN DE ARCHIVOS DE ENTRENAMIENTO Y VALIDACIÓN**

In [ ]:
!python '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/SCRIPTS/partition_dataset.py' -x -i /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/ARCHIVOS -r 0.1

# **GENERACIÓN DE LOS TFRECORDS**

In [ ]:
!python '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/SCRIPTS/generate_tfrecord.py' -x /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/ARCHIVOS/train -l /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/LABELMAP/label_map.pbtxt -o /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/RECORD-TRAIN/train.record

In [ ]:
!python '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/SCRIPTS/generate_tfrecord.py' -x /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/ARCHIVOS/test -l /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/LABELMAP/label_map.pbtxt -o /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/RECORD-TEST/test.record

# **DESCARGA Y CONFIGURACIÓN DEL MODELO**

In [ ]:
cd /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/PRE-TRAINED

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz

In [ ]:
!tar xvzf /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/TRAIN/PRE-TRAINED/efficientdet_d4_coco17_tpu-32.tar.gz

# **COMIENZO DEL ENTRENAMIENTO**

In [ ]:
!python /usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/SCRIPTS/model_main_tf2.py --model_dir=/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/MIS-MODELOS/myEfficient --pipeline_config_path=/usr/share/Data2/objectos_pequeños/PRINCIPAL/RENTRENAMIENTO/MIS-MODELOS/myEfficient/ssd_efficientdet_d4_1024x1024_coco17_tpu-32.config

# **EVALUACIÓN DE LOS MODELOS CKPT**

In [ ]:
PATH_TO_CFG = "/home/ivangarcia/Desktop/MisModelos/efficientd4mod/ssd_efficientdet_d4_1024x1024_coco17_tpu-32.config"
PATH_TO_CKPT = "/home/ivangarcia/Desktop/MisModelos/efficientd4mod"

print('Loading model... ', end='')
start_time = time.time()

configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [ ]:
def make_checkpoint_final(image_path,counter,image_save):

    image_np = load_image_into_numpy_array(image_path)
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections = image_np.copy()
    
    boxes_filter = []
    scores_filter = []
    classes_filter = []

    for item in range(len(detections['detection_boxes'])):
        boxes_filter.append(detections['detection_boxes'][item].tolist())
        scores_filter.append(detections['detection_scores'][item])
        classes_filter.append(detections['detection_classes'][item])


    detections['detection_boxes'] = np.array(boxes_filter)
    detections['detection_classes'] = np.array(classes_filter)
    detections['detection_scores'] = np.array(scores_filter)

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.4,
            agnostic_mode=False)

    plt.figure()
    plt.axis('off')
    nombre = counter
    plt.imshow(image_np_with_detections)
    plt.savefig(image_save+nombre,  dpi=dpi ,bbox_inches='tight',pad_inches = 0)

    plt.clf()

    boxes = detections['detection_boxes']
    scores = detections['detection_scores'],
    clases_detected = detections['detection_classes']
    img = cv2.imread(image_path)

    boxes = detections['detection_boxes']
    scores = detections['detection_scores'],
    clases_detected = detections['detection_classes']

    min_score_thresh = .3
    true_boxes  = boxes[scores[0] > min_score_thresh]
    true_scores = scores[0][scores[0] > min_score_thresh]
    true_clases = clases_detected[scores[0] > min_score_thresh]

    salida = []
    salida.append(true_boxes.tolist())
    salida.append(true_scores.tolist())
    salida.append(true_clases.tolist()) 

    image = Image.open(image_path)
    width, height = image.size

    salida.append(width)
    salida.append(height)

    return salida


In [ ]:
import re
imagenes_dir2 = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images'
contenido2 = os.listdir(imagenes_dir2)
imagenes_dir2 = []
counter = 0
result = []
tiempos_procesar = []

out = []
concon = 0
for fichero2 in contenido2:
  print(concon)
  concon = concon +1
  x = re.search("[1-9]+[0-9]*", fichero2)
  id = x.group(0)
  print(id)
  image_path_gg = "/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/images/{}".format(fichero2)
  start_time = time()
  salida = make_checkpoint_final(image_path_gg,fichero2,'/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/')
  elapsed_time = time() - start_time
  tiempos_procesar.append(elapsed_time)
  result = []
  converted_num = int(id)
  converted_num = diccionario_ids[fichero2]
  print(fichero2, " ",converted_num)

  width = salida[3]
  height = salida[4]

  boxes2 = []

  for box in salida[0]:
    ymin = int(box[0]*height)
    xmin = int(box[1]*width)
    ymax = int(box[2]*height)
    xmax = int(box[3]*width) 


    box_new = []
    box_new.append(xmin)
    box_new.append(ymin)
    box_new.append(xmax-xmin)
    box_new.append(ymax-ymin)
  

    boxes2.append(box_new)
   


  result.extend(
        [
            {
                "image_id": converted_num,
                "category_id": salida[2][k],
                "bbox": box,
                "score": salida[1][k],
            }
            for k, box in enumerate(boxes2)
        ]
  )
  
  for sample in result:    
        out.append(sample)#detection result including 

with open('/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/test_data_normal.json', 'w', encoding='utf-8') as file:
  json.dump(out, file, ensure_ascii=False)

In [ ]:
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

annFile = '/usr/share/Data2/objectos_pequeños/FOTOSYEVAULACION/video1/annotations/instances_default.json'
cocoGt=COCO(annFile)

resFile = '/usr/share/Data2/objectos_pequeños/PRINCIPAL/RESULTADOS/test_data_normal.json'
cocoDt=cocoGt.loadRes(resFile)

dts = json.load(open(resFile,'r'))
imgIds = [imid['image_id'] for imid in dts]
imgIds = sorted(list(set(imgIds)))


cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.params.catIds = [3] 
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()